In [ ]:
from datosipc import datosIPC

In [ ]:
p = datosIPC(2023, 1, dbBackup=True)

In [ ]:
p.inflacion_CA_RD_MEX()


In [ ]:
from sqline import sqlINE

In [ ]:
p = sqlINE(2023, 2, dbBackup=1)

In [ ]:
datos = p.serie_historica("interanual")
datos

In [ ]:
max(datos, key=lambda x: x[1])

In [ ]:
min(datos, key=lambda x: x[1])

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.DataFrame(p.serie_historica("interanual"), columns=["Fecha", "interanual"])

In [ ]:
df2 = pd.DataFrame(p.serie_historica("intermensual"), columns=["Fecha", "intermensual"])

In [ ]:
df = pd.merge(df1, df2, on="Fecha", how="right").fillna("-")
df

In [ ]:
df.to_latex("tabla_prueba.tex", index=False)

In [ ]:
with open('tabla_prueba.tex', 'w') as f:
    f.write(df.style.hide_index().set_caption('Tabla de prueba').set_table_styles([{'selector': 'caption', 'props': [('text-align', 'center')]}]).to_latex())

In [ ]:
with open('tabla_prueba.tex', 'w') as f:
    f.write(df.to_latex(index=False, longtable=True, caption='Tabla de prueba', header=["Fecha", "Mensual", "Anual"], 
                        column_format='|c|c|c|', escape=False).replace('\\hline', '\\cline{1-3}').replace('\\endfirsthead', '\\hline\n\\endhead\n'))

In [ ]:
with open('tabla_prueba.tex', 'w') as f:
    f.write(df.to_latex(index=False, longtable=True, caption='Tabla de prueba', 
                        column_format='|c|c|c|', escape=False, 
                        header=['Fecha', 'Interanual', 'Intermensual']).replace('\\hline', '\\cline{1-3}').replace('\\endfirsthead', '\\hline\n\\endhead\n'))

In [ ]:
def export_to_longtable(df, filename, caption, column_format='|c|c|', header=True):
    with open(filename, 'w+') as f:
        if header:
            f.write(df.head(1).to_latex(index=False, longtable=False, column_format=column_format, escape=False, header=header).replace('\\hline', '\\cline{1-2}').replace('\\endfirsthead', '\\hline\n\\endhead\n'))
        f.write(df.to_latex(index=False, longtable=True, column_format=column_format, escape=False, header=False).replace('\\hline', '\\cline{1-2}').replace('\\endfirsthead', '\\hline\n\\endhead\n'))
        f.seek(0)
        contents = f.read()
        f.seek(0)
        f.write('\\begin{longtable}{' + column_format + '}\n')
        f.write('\\caption{' + caption + '} \\\\\n')
        f.write('\\hline\n')
        f.write(contents)
        f.write('\\end{longtable}\n')

    
export_to_longtable(df, 'tabla_prueba.tex', 'Tabla de prueba')

In [ ]:
def export_to_longtable(df, filename, caption, column_format='c'*len(df.columns), header=True):
    """
    Exporta un DataFrame a un archivo .tex en formato longtable.
    
    Parameters:
    df (pandas.DataFrame): DataFrame a exportar.
    filename (str): Nombre del archivo .tex a crear.
    caption (str): Título de la tabla.
    column_format (str, optional): Formato de las columnas. Por defecto es 'c'*len(df.columns).
    header (bool, optional): Si se incluye la primera fila como encabezado. Por defecto es True.
    """
    with open(filename, 'w') as f:
        f.write('\\begin{longtable}{' + column_format + '}\n')
        if header:
            f.write('\\caption{' + caption + '}\\label{' + filename[:-4] + '-label}\\tabularnewline\n')
            f.write('\\toprule\n')
            for col in df.columns:
                f.write('\\textbf{' + col + '}')
                if col != df.columns[-1]:
                    f.write(' & ')
                else:
                    f.write(' \\\\\n')
            f.write('\\midrule\n')
            f.write('\\endfirsthead\n')
            f.write('\\multicolumn{' + str(len(df.columns)) + '}{c}{{Tabla \\thetable{} continuación de la página anterior.}} \\\\\n')
            f.write('\\toprule\n')
            for col in df.columns:
                f.write('\\textbf{' + col + '}')
                if col != df.columns[-1]:
                    f.write(' & ')
                else:
                    f.write(' \\\\\n')
            f.write('\\midrule\n')
            f.write('\\endhead\n')
        else:
            f.write('\\caption{' + caption + '}\\label{' + filename[:-4] + '-label}\\tabularnewline\n')
            f.write('\\toprule\n')
        for i, row in df.iterrows():
            for j, val in enumerate(row):
                if pd.isna(val):
                    val = '-'
                if isinstance(val, float):
                    f.write('{:.6f}'.format(val))
                else:
                    f.write(str(val))
                if j != len(row) - 1:
                    f.write(' & ')
                else:
                    f.write(' \\\\\n')
            f.write('\\midrule\n')
        f.write('\\end{longtable}\n')


In [ ]:
def export_to_longtable(df, filename, caption, column_format='c'*len(df.columns), header=True, decimals=2):
    """
    Exporta un dataframe a un longtable de LaTeX en un archivo.
    
    Parameters:
    df (pandas.DataFrame): DataFrame a exportar.
    filename (str): Nombre del archivo a guardar.
    caption (str): Leyenda de la tabla.
    column_format (str): Cadena que describe el formato de las columnas.
    header (bool): Si True, muestra el encabezado de la tabla.
    decimals (int): Cantidad de decimales para los números. Si el valor no es numérico, se muestra tal cual.
    """
    with open(filename, 'w', encoding="utf-8") as f:
        f.write('\\begin{longtable}{' + column_format + '}\n')
        f.write('\\caption{' + caption + '}\\label{tab:' + filename[:-4] + '}\\\\\n')
        f.write('\\toprule\n')
        
        # Escribir el encabezado
        if header:
            f.write(' & '.join([col.replace('_', '\\_') for col in df.columns]))
            f.write('\\\\\\midrule\n')
        
        f.write('\\endfirsthead\n')
        f.write('\\multicolumn{' + str(len(df.columns)) + '}{c}{{\\bfseries \\tablename\\ \\thetable{} -- '
                'Continuación de la página anterior}}\\\\\n')
        f.write('\\toprule\n')
        if header:
            f.write(' & '.join([col.replace('_', '\\_') for col in df.columns]))
            f.write('\\\\\\midrule\n')
        f.write('\\endhead\n')
        f.write('\\midrule\n')
        f.write('\\multicolumn{' + str(len(df.columns)) + '}{r}{{Continúa en la siguiente página}}\\\\\n')
        f.write('\\endfoot\n')
        f.write('\\bottomrule\n')
        f.write('\\endlastfoot\n')
        
        # Escribir los datos
        import re
        for _, row in df.iterrows():
            values = []
            for value in row:
                if pd.isna(value):
                    values.append('')
                elif isinstance(value, (int, float)):
                    valor = round(value, decimals)
                    if re.match(r"^[-]0\.0*$", str(valor)):
                        valor = int(valor)
                    valor = '{:.{}f}'.format(valor, decimals)
                    values.append(valor)
                else:
                    values.append(str(value))
            f.write(' & '.join(values))
            f.write('\\\\\n')
        
        f.write('\\end{longtable}\n')


In [ ]:
export_to_longtable(df, 'tabla_prueba2.tex', 'Tabla variación interanual e intermnesual', decimals=2) 


In [1]:
from datosipc import datosIPC

In [3]:
prueba = datosIPC(2023, 3, dbBackup=1)

In [5]:
prueba.serie_fuentes()

([('Mar-2022', 6665),
  ('Abr-2022', 6683),
  ('May-2022', 6020),
  ('Jun-2022', 6733),
  ('Jul-2022', 7115),
  ('Ago-2022', 6090),
  ('Sep-2022', 6777),
  ('Oct-2022', 7004),
  ('Nov-2022', 7165),
  ('Dic-2022', 7721),
  ('Ene-2023', 8631),
  ('Feb-2023', 8067),
  ('Mar-2023', 8531)],
 'La cantidad de fuentes consultadas en marzo 2023 es de 8,531. La mayor cantidad de fuentes consultadas fue en el mes de enero 2023 con una cantidad de 8,631 y la menor se encuentra en el mes de mayo 2022 con una cantidad de 6,020.')

In [6]:
prueba.serie_precios()

([('Mar-2022', 34061),
  ('Abr-2022', 33671),
  ('May-2022', 31533),
  ('Jun-2022', 34240),
  ('Jul-2022', 34901),
  ('Ago-2022', 32351),
  ('Sep-2022', 35369),
  ('Oct-2022', 35894),
  ('Nov-2022', 38208),
  ('Dic-2022', 41557),
  ('Ene-2023', 44692),
  ('Feb-2023', 44106),
  ('Mar-2023', 46054)],
 'La cantidad de precios diligenciados en marzo 2023 es de 46,054. La mayor cantidad de precios diligenciados fue en el mes de marzo 2023 con una cantidad de 46,054 y la menor se encuentra en el mes de mayo 2022 con una cantidad de 31,533.')